# Wide and Deep model

In [2]:
import sys
import pandas as pd
import numpy as np

import tensorflow as tf
import sklearn

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
sns.set_style("darkgrid", {"axes.facecolor": ".95"})
sns.set_context("paper")

In [3]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

# Model 1

<img src="images/wide-and-deep-01.png" width="400px">

In [5]:
input_A = tf.keras.layers.Input(shape=[5], name="wide_input")
input_B = tf.keras.layers.Input(shape=[6], name="deep_input")
hidden1 = tf.keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([input_A, hidden2])
output = tf.keras.layers.Dense(1, name="output")(concat)
model = tf.keras.models.Model(inputs=[input_A, input_B], outputs=[output])

In [12]:
model.compile(loss="mse", optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3))

X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

history = model.fit((X_train_A, X_train_B), y_train, epochs=20,
                    validation_data=((X_valid_A, X_valid_B), y_valid))
mse_test = model.evaluate((X_test_A, X_test_B), y_test)
y_pred = model.predict((X_test_A, X_test_B))

Epoch 1/20
363/363 [==============================] - 0s 888us/step - loss: 0.4312 - val_loss: 0.4177
Epoch 2/20
363/363 [==============================] - 0s 718us/step - loss: 0.4289 - val_loss: 0.4109
Epoch 3/20
363/363 [==============================] - 0s 722us/step - loss: 0.4264 - val_loss: 0.4190
Epoch 4/20
363/363 [==============================] - 0s 725us/step - loss: 0.4241 - val_loss: 0.4123
Epoch 5/20
363/363 [==============================] - 0s 737us/step - loss: 0.4222 - val_loss: 0.4104
Epoch 6/20
363/363 [==============================] - 0s 743us/step - loss: 0.4203 - val_loss: 0.4157
Epoch 7/20
363/363 [==============================] - 0s 746us/step - loss: 0.4179 - val_loss: 0.4139
Epoch 8/20
363/363 [==============================] - 0s 740us/step - loss: 0.4165 - val_loss: 0.3995
Epoch 9/20
363/363 [==============================] - 0s 747us/step - loss: 0.4149 - val_loss: 0.3970
Epoch 10/20
363/363 [==============================] - 0s 730us/step - loss: 0.413

In [14]:
from sklearn.metrics import mean_absolute_percentage_error

mean_absolute_percentage_error(y_test, y_pred)

0.2687186708204187

----

# Model 3

<img src="images/wide-and-deep-03.png" width="400px">

In [ ]:
input_A = tf.keras.layers.Input(shape=[5], name="wide_input")
input_B = tf.keras.layers.Input(shape=[6], name="deep_input")
hidden1 = tf.keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)
concat = tf.keras.layers.concatenate([input_A, hidden2])
output = tf.keras.layers.Dense(1, name="main_output")(concat)
aux_output = tf.keras.layers.Dense(1, name="aux_output")(hidden2)
model = tf.keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])